In [ ]:
from urllib import request,parse
import ssl
from http import cookiejar
from http.cookiejar import MozillaCookieJar

**urlopen**

向指定url发送请求，包括get、post两种方式。区别是是否有data参数。
参数：

    (1)url：string或Request对象
    (2)data（默认空）：是伴随 url 提交的数据（比如要post的数据），同时 HTTP 请求将从 "GET"方式 改为 "POST"方式。
    (3)timeout：超时限制。

In [ ]:
# 向指定的url发送请求，并返回服务器响应的类文件对象
response = request.urlopen("http://www.baidu.com")

# 类文件对象支持 文件对象的操作方法，如read()方法读取文件全部内容，返回字符串
#获得的数据会是bytes的二进制格式，所以需要decode()一下，转换成字符串格式。
html = response.read().decode()
print(html)

**Request**

如果需要执行更复杂的操作，比如增加HTTP报头，必须创建一个 Request 实例来作为urlopen()的参数；
新建Request实例，所需参数： 

    (1)url 
    (2)data（默认空）：是伴随 url 提交的数据（比如要post的数据），同时 HTTP 请求将从 "GET"方式 改为 "POST"方式。
    (3)headers（默认空）：是一个字典，包含了需要发送的HTTP报头的键值对。

In [ ]:
# url 作为Request()方法的参数，构造并返回一个Request对象
req = request.Request("http://www.baidu.com")

# Request对象作为urlopen()方法的参数，发送给服务器并接收响应
response = request.urlopen(req)
print(response)
print(response.read().decode())

**伪装浏览器：User-Agent**

直接给一个网站发送请求的话，确实略有些唐突了，就好比，人家每家都有门，你以一个路人的身份直接闯进去显然不是很礼貌。
而且有一些站点不喜欢被程序（非人为访问）访问，有可能会拒绝你的访问请求。

但是如果我们用一个合法的身份去请求别人网站，显然人家就是欢迎的，所以我们就应该给我们的这个代码加上一个身份，就是所谓的User-Agent头 。

浏览器 就是互联网世界上公认被允许的身份，如果我们希望我们的爬虫程序更像一个真实用户，那我们第一步，就是需要伪装成一个被公认的浏览器。
用不同的浏览器在发送请求的时候，会有不同的User-Agent头。 
urllib默认的User-Agent头为：Python-urllib/x.y（x和y是Python主版本和次版本号,例如 Python-urllib/2.7）

`# used in User-Agent header sent

__version__ = '%d.%d' % sys.version_info[:2]

client_version = "Python-urllib/%s" % __version__

self.addheaders = [('User-agent', client_version)]`

In [ ]:
url = "http://www.baidu.com"
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36 Edg/83.0.478.44"
}

req = request.Request(url,headers=headers)

repsonse = request.urlopen(req)
print(repsonse)
print(repsonse.read().decode())

**对header的操作**

    添加：Request.add_header(self, key, val)
    删除：Request.remove_header(self, header_name)
    获取：Request.get_header(self, header_name)

**response对象**

除了支持文件操作的方法外，还支持如下方法：

    * geturl() - 返回实际数据的实际url，防止重定向问题

    * info() - 返回服务器响应的HTTP报头, such as headers, in the
      form of an email.message_from_string() instance (see Quick Reference to
      HTTP Headers)

    * getcode() - 返回HTTP响应码.  Raises URLError
      on errors.

In [ ]:
url = "http://www.baidu.com"
ua= "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36 Edg/83.0.478.44"

req = request.Request(url)
req.add_header("User-Agent",ua)

repsonse = request.urlopen(req)
print(repsonse)

In [ ]:
from urllib import parse
d1 = {"wd":"香蕉"}
# 编码：编码成 URL编码格式，然后做为url的一部分，或者作为参数传到Request对象中。
d2 = parse.urlencode(d1)
print(d2)  # wd=%E9%A6%99%E8%95%89
# 解码，把 URL编码字符串，转换回原先字符串
d3 = parse.unquote(d2)
print(d3)

In [ ]:
url = "http://www.baidu.com/s"
word = parse.urlencode({"wd":"苹果"})
newurl = url + "?" + word
ua= "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36 Edg/83.0.478.44"

req = request.Request(newurl)
req.add_header("User-Agent",ua)

repsonse = request.urlopen(req)
print(repsonse)

**SSL**

如果网站的SSL证书是经过CA认证的，则能够正常访问，如：https://www.baidu.com/等。
如果SSL证书验证不通过，或者操作系统不信任服务器的安全证书。
这种情况需要单独处理SSL证书，让程序忽略SSL证书验证错误。

In [ ]:
# 单独处理SSL证书
url = "https://www.12306.cn/mormhweb/"  #举例
headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36 Edg/83.0.478.44",
}

# 表示忽略未经核实的SSL证书认证
context = ssl._create_unverified_context()

req = request.Request(url, headers = headers)

response = request.urlopen(req,context = context)

print(response.read().decode())

**CA**

CA(Certificate Authority)是数字证书认证中心的简称，是指发放、管理、废除数字证书的受信任的第三方机构，如北京数字认证股份有限公司、上海市数字证书认证中心有限公司等...
CA的作用是检查证书持有者身份的合法性，并签发证书，以防证书被伪造或篡改，以及对证书和密钥进行管理。
现实生活中可以用身份证来证明身份， 那么在网络世界里，数字证书就是身份证。

**Handler处理器 和 自定义Opener**

一直都在使用的urlopen，它是一个特殊的opener（也就是模块帮我们构建好的）。
但是基本的urlopen()方法不支持代理、cookie等其他的HTTP/HTTPS高级功能。所以要自定义：

    使用相关的 Handler处理器 来创建特定功能的处理器对象
    然后通过 urllib2.build_opener()方法使用这些处理器对象，创建自定义opener对象；
    使用自定义的opener对象，调用open()方法发送请求。
    
如果程序里所有的请求都使用自定义的opener，可以使用urllib2.install_opener() 将自定义的 opener 对象 定义为 全局opener，
表示如果之后凡是调用urlopen，都将使用这个opener（根据自己的需求来选择）

In [ ]:
# 构建一个HTTPHandler 处理器对象，支持处理HTTP请求
http_handler = request.HTTPHandler()

# 构建一个HTTPHandler 处理器对象，支持处理HTTP请求，同时开启Debug Log，debuglevel 值默认 0
#http_handler = request.HTTPHandler(debuglevel=1)

# 调用build_opener()方法，创建支持处理HTTP请求的opener对象
opener = request.build_opener(http_handler)

response = opener.open("http://www.baidu.com/")
print(response.read().decode())

**ProxyHandler处理器（代理设置）**

很多网站会检测某一段时间某个IP的访问次数(通过流量统计，系统日志等)，如果访问次数多的不像正常人，它会禁止这个IP的访问。

所以我们可以设置一些代理服务器，每隔一段时间换一个代理，就算IP被禁止，依然可以换个IP继续爬取。

可以通过ProxyHandler来设置使用代理服务器

In [ ]:
proxySwitch = True #定义一个代理开关

http_proxyhandler = request.ProxyHandler({"http" : "124.88.67.81:80"})
# http_proxyhandler = request.ProxyHandler({"http" : "name:password@124.88.67.81:80"})  # 私密代理验证1
null_proxyhandler = request.ProxyHandler()

if proxySwitch :
    opener = request.build_opener(http_proxyhandler)
else:
    opener = request.build_opener(null_proxyhandler)
    
# 1. 如果这么写，只有使用opener.open()方法发送请求才使用自定义的代理，而urlopen()则不使用自定义代理。
response = opener.open("http://www.baidu.com/")

# 2. 如果这么写，就是将opener应用到全局，之后所有的，不管是opener.open()还是urlopen() 发送请求，都将使用自定义代理。
# request.install_opener(opener)
# response = urlopen(request)
print(response.read().decode())

In [ ]:
# 随机获取代理
import random
proxy_list = [
    {"http" : "124.88.67.81:80"},
    {"http" : "124.88.67.81:80"},
    {"http" : "124.88.67.81:80"},
    {"http" : "124.88.67.81:80"},
    {"http" : "124.88.67.81:80"}
]
proxy = random.choice(proxy_list)

httpproxy_handler = request.ProxyHandler(proxy)

opener = request.build_opener(httpproxy_handler)

request = request.Request("http://www.baidu.com/")

response = opener.open(request)
print(response.read().decode())

In [ ]:
# 使用私密代理2

# 私密代理授权的账户
user = "mr_mao_hacker"
# 私密代理授权的密码
passwd = "sffqry9r"
# 私密代理 IP
proxyserver = "61.158.163.130:16816"

# 1. 构建一个密码管理对象，用来保存需要处理的用户名和密码
passwdmgr = request.HTTPPasswordMgrWithDefaultRealm()

# 2. 添加账户信息，第一个参数realm是与远程服务器相关的域信息，一般没人管它都是写None，后面三个参数分别是 代理服务器、用户名、密码
passwdmgr.add_password(None, proxyserver, user, passwd)

# 3. 构建一个代理基础用户名/密码验证的ProxyBasicAuthHandler处理器对象，参数是创建的密码管理对象
#   注意，这里不再使用普通ProxyHandler类了
proxyauth_handler = request.ProxyBasicAuthHandler(passwdmgr)

# 4. 通过 build_opener()方法使用这些代理Handler对象，创建自定义opener对象，参数包括构建的 proxy_handler 和 proxyauth_handler
opener = request.build_opener(proxyauth_handler)

# 5. 构造Request 请求
request = request.Request("http://www.baidu.com/")

# 6. 使用自定义opener发送请求
response = opener.open(request)

# 7. 打印响应内容
print(response.read().decode())

有些Web服务器（包括HTTP/FTP等）访问时，需要进行用户身份验证，爬虫直接访问会报HTTP 401 错误，表示访问身份未经授权：

    HTTP Error 401: Unauthorized
    
如果我们有客户端的用户名和密码，我们可以通过下面的方法去访问爬取：

In [ ]:
test = "test"
password = "123456"
webserver = "192.168.21.52"

# 构建一个密码管理对象，可以用来保存和HTTP请求相关的授权账户信息
passwordMgr = request.HTTPPasswordMgrWithDefaultRealm()

# 添加授权账户信息，第一个参数realm如果没有指定就写None，后三个分别是站点IP，账户和密码
passwordMgr.add_password(None, webserver, test, password)

# HTTPBasicAuthHandler() HTTP基础验证处理器类
httpauth_handler = request.HTTPBasicAuthHandler(passwordMgr)

# 处理代理基础验证相关的处理器类
proxyauth_handler = request.ProxyBasicAuthHandler(passwordMgr)

# 构建自定义opener
opener = request.build_opener(httpauth_handler, proxyauth_handler)

request = request.Request("http://192.168.21.52/")

# 用授权验证信息
response = opener.open(request)
print(response.read().decode())

**Cookie**

Cookie 是指某些网站服务器为了辨别用户身份和进行Session跟踪，而储存在用户浏览器上的文本文件，Cookie可以保持登录信息到用户下次与服务器的会话。

**Cookie原理**

HTTP是无状态的面向连接的协议, 为了保持连接状态, 引入了Cookie机制 Cookie是http消息头中的一种属性，包括：

        Cookie名字（Name）
        Cookie的值（Value）
        Cookie的过期时间（Expires/Max-Age）
        Cookie作用路径（Path）
        Cookie所在域名（Domain），
        使用Cookie进行安全连接（Secure）。

前两个参数是Cookie应用的必要条件，另外，还包括Cookie大小（Size，不同浏览器对Cookie个数及大小限制是有差异的）。
Cookie由变量名和值组成，根据 Netscape公司的规定，Cookie格式如下：

        Set－Cookie: NAME=VALUE；Expires=DATE；Path=PATH；Domain=DOMAIN_NAME；SECURE

In [ ]:
# 获取一个有登录信息的Cookie模拟登陆
# 抓包获取cookie
url = "https://home.cnblogs.com/u/1094968"
headers = {
    "Accept": "text/plain, */*; q=0.01",
    "X-Requested-With": "XMLHttpRequest",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36 Edg/83.0.478.44",
    "Accept-Language": "zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6",
    "cookie":""
    }

req = request.Request(url,headers=headers)
repsonse = request.urlopen(req)
print(repsonse.read().decode())

**CookieJar 和 HTTPCookieProcessor处理器**

    HTTPCookieProcessor处理器：主要作用是处理这些cookie对象，并构建handler对象。

    CookieJar：管理HTTP cookie值、存储HTTP请求生成的cookie、向传出的HTTP请求添加cookie的对象。整个cookie都存储在内存中，对CookieJar实例进行垃圾回收后cookie也将丢失。
    
    FileCookieJar (filename,delayload=None,policy=None)：从CookieJar派生而来，用来创建FileCookieJar实例，检索cookie信息并将cookie存储到文件中。filename是存储cookie的文件名。delayload为True时支持延迟访问访问文件，即只有在需要时才读取文件或在文件中存储数据。
    
    MozillaCookieJar (filename,delayload=None,policy=None)：# 创建MozillaCookieJar(有load实现)实例对象
    
    LWPCookieJar (filename,delayload=None,policy=None)：从FileCookieJar派生而来，创建与libwww-perl标准的 Set-Cookie3 文件格式兼容的FileCookieJar实例。

其实大多数情况下，我们只用CookieJar()，如果需要和本地文件交互，就用 MozillaCookjar() 或 LWPCookieJar()

In [ ]:
# 获取Cookie，并保存到CookieJar()对象中

# 构建一个CookieJar对象实例来保存cookie
cjar = cookiejar.CookieJar()

# 使用HTTPCookieProcessor()来创建cookie处理器对象，参数为CookieJar()对象
handler = request.HTTPCookieProcessor(cjar)

opener = request.build_opener(handler)

# 以get方法访问页面，访问之后会自动保存cookie到cookiejar中
opener.open("http://www.baidu.com")

cookieStr = ""
for item in cjar:
    cookieStr = cookieStr + item.name + "=" + item.value + ";"

## 舍去最后一位的分号
print(cookieStr[:-1])

In [ ]:
# 访问网站获得cookie，并把获得的cookie保存在cookie文件中

# 保存cookie的本地磁盘文件名
filename = 'cookie.txt'

# 声明一个MozillaCookieJar(有save实现)对象实例来保存cookie，之后写入文件
cjar = MozillaCookieJar(filename)

# 使用HTTPCookieProcessor()来创建cookie处理器对象，参数为CookieJar()对象
handler = request.HTTPCookieProcessor(cjar)

# 通过 build_opener() 来构建opener
opener = request.build_opener(handler)

# 创建一个请求，原理同urllib2的urlopen
response = opener.open("http://www.baidu.com")

# 保存cookie到本地文件
cjar.save()

In [ ]:
#从文件中获取cookies，做为请求的一部分去访问
# 创建MozillaCookieJar(有load实现)实例对象
cjar = MozillaCookieJar()

# 从文件中读取cookie内容到变量
cjar.load('cookie.txt')

# 使用HTTPCookieProcessor()来创建cookie处理器对象，参数为CookieJar()对象
handler = request.HTTPCookieProcessor(cookiejar)

# 通过 build_opener() 来构建opener
opener = request.build_opener(handler)

responses = opener.open("http://www.baidu.com")

